In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import re
import spacy
from textblob import TextBlob
from gensim import corpora, models
import numpy as np
from gensim.models import Word2Vec
from textstat import flesch_kincaid_grade

In [2]:
df = pd.read_excel('../data/cleaned_unlabeled_dataset_2.xlsx')
df

,IDText,Text,clean_text
0,TXT0001,Lu mau org2 pro-demokrasi di negara ini bisa p...,org2 pro-demokrasi negara kesempatan ngelawan ...
1,TXT0002,Prabowo ditanya soal hutang luar negeri dia me...,prabowo ditanya hutang negeri menjawab hutang ...
2,TXT0003,kiki_daliyo Ganjar Pranowo itulah beliau soso...,kiki_daliyo ganjar pranowo sosok mengagunhkan ...
3,TXT0004,@kumparan Prabowo Gibran yang bisa melakukan i...,prabowo gibran melakukan kesejahteraan rakyat
4,TXT0005,@sniperruben45 @uda_zulhendra @ainunnajib Lah ...,nyambung junjungan elu aomkmkmkmk ngomongin et...
...,...,...,...
995,TXT0996,"Bikin bangga deh, Ganjar-Mahfud mau alokasikan...","bikin bangga deh, ganjar-mahfud alokasikan bel..."
996,TXT0997,Pak Jokowi sebelum pilpres 2024 berbesar hati ...,jokowi pilpres 2024 berbesar hati merangkul pr...
997,TXT0998,@datuakrajoangek Sbaiknya si gemot nga usah ik...,sbaiknya si gemot nga ikutan debat deh..nga ma...
998,TXT0999,kebiasaan merembuk atau bermusyawarah jadi gay...,kebiasaan merembuk bermusyawarah gaya kepemimp...


In [18]:
# cari hstag yang paling banyak muncul
def get_hastag(text):
    return re.findall(r"#(\w+)", text)

df['hastag'] = df['Text'].apply(get_hastag)
df['hastag'] = df['hastag'].apply(lambda x: ' '.join(x))
df['hastag'] = df['hastag'].apply(lambda x: x.lower())
df['hastag'] = df['hastag'].apply(lambda x: x.split())

hastag = df['hastag'].tolist()
hastag = [item for sublist in hastag for item in sublist]
hastag = pd.Series(hastag)
hastag = hastag.value_counts()
hastag = hastag[:10]
hastag

ganjarmahfud2024            185
l3bihbaik                   119
jnk                         109
ganjarpranowopilihanumat    108
ganjarmahfudrebound          89
coblos3                      80
ganjarmahfudm3nangdebat      70
3gm                          62
02melanjutkan                53
3mandatganjarmahfud          51
dtype: int64

In [15]:
from extraction_module import get_top_words, get_lda_topics, get_lda_topics, get_top_words_percentage

In [10]:
words_freq_percentage, features = get_top_words_percentage(df, n=100)

print(words_freq_percentage)
print(features)

# simpan hasil ekstraksi ke dalam file json
import json
with open('words_freq_percentage.json', 'w') as f:
    json.dump(words_freq_percentage, f)


{'ganjar': 0.027270421106037543, 'prabowo': 0.02139353965837984, 'anies': 0.01830711990529342, 'mahfud': 0.014248266531371553, 'pranowo': 0.00837138508371385, 'ganjarmahfud2024': 0.008075427025198714, 'indonesia': 0.007821748689328598, 'capres': 0.007314392017588364, 'dalam': 0.007102993404363267, 'jokowi': 0.00532724505327245, 'rakyat': 0.005115846440047353, 'gibran': 0.0050735667174023336, 'debat': 0.0050735667174023336, 'l3bihbaik': 0.005031286994757314, 'presiden': 0.0047776086588871974, 'jnk': 0.0046930492135971585, 'ganjarpranowopilihanumat': 0.0045662100456621, 'hebat': 0.004523930323017081, 'korupsi': 0.004270251987146964, 'happy': 0.003889734483341789, 'demokrasi': 0.003889734483341789, 'new': 0.0038474547606967696, 'year': 0.0037628953154067307, 'ganjarmahfudrebound': 0.0037628953154067307, 'ekonomi': 0.0037206155927617113, 'cawapres': 0.003593776424826653, 'anggaran': 0.0034669372568915946, 'coblos3': 0.0033823778116015557, 'negara': 0.0033400980889565363, '2024': 0.00334009

In [ ]:
words_freq_kata, features = get_top_words(df, n=100)

print(words_freq_kata)

# simpan jadi dictionary
words_freq_kata = dict(words_freq_kata)

# simpan jadi file
import json
with open('top_words.json', 'w') as f:
    json.dump(words_freq_kata, f)
    

{'ganjar': 645, 'prabowo': 506, 'anies': 433, 'mahfud': 337, 'pranowo': 198, 'ganjarmahfud2024': 191, 'indonesia': 185, 'capres': 173, 'dalam': 168, 'jokowi': 126, 'rakyat': 121, 'gibran': 120, 'debat': 120, 'l3bihbaik': 119, 'presiden': 113, 'jnk': 111, 'ganjarpranowopilihanumat': 108, 'hebat': 107, 'korupsi': 101, 'happy': 92, 'demokrasi': 92, 'new': 91, 'year': 89, 'ganjarmahfudrebound': 89, 'ekonomi': 88, 'cawapres': 85, 'anggaran': 82, 'coblos3': 80, 'negara': 79, '2024': 79, 'kesejahteraan': 78, 'md': 76, 'ini': 75, 'baswedan': 72, 'pasangan': 70, 'ganjarmahfudm3nangdebat': 70, 'pilpres': 69, 'gubernur': 67, 'pertahanan': 67, '3gm': 62, 'nomor': 62, 'masyarakat': 61, 'pertumbuhan': 59, 'program': 57, 'pemimpin': 56, 'pemerintah': 56, '02melanjutkan': 53, 'etika': 52, 'bikin': 51, '3mandatganjarmahfud': 51, 'pendukung': 51, 'politik': 48, 'pemerintahan': 45, 'memang': 45, 'anak': 44, 'masuk': 44, 'hukum': 43, 'ganjarmahfudm3nang': 43, 'lain': 42, 'debatcapres': 42, 'menhan': 42, '

In [6]:
topic_features, topics = get_lda_topics(df, num_topics=10)
print(topics)
print(topic_features)


[(0, '0.018*"prabowo" + 0.014*"anies" + 0.005*"negara" + 0.004*"debat" + 0.003*"maka"'), (1, '0.029*"prabowo" + 0.011*"#02melanjutkan" + 0.010*"anies" + 0.010*"gibran" + 0.008*"#indonesiasentris"'), (2, '0.063*"ganjar" + 0.029*"mahfud" + 0.019*"hebat" + 0.016*"pranowo" + 0.015*"#coblos3"'), (3, '0.024*"#l3bihbaik" + 0.024*"#ganjarmahfud2024" + 0.022*"year" + 0.022*"happy" + 0.022*"new"'), (4, '0.019*"anies" + 0.017*"prabowo" + 0.006*"presiden" + 0.005*"dalam" + 0.004*"pilpres"'), (5, '0.029*"ganjar" + 0.022*"#ganjarpranowopilihanumat" + 0.022*"pranowo" + 0.020*"capres" + 0.017*"#jnk"'), (6, '0.011*"demokrasi" + 0.008*"ganjar" + 0.008*"memilih" + 0.007*"-" + 0.007*"pranowo."'), (7, '0.018*"anies" + 0.008*"debat" + 0.005*"prabowo" + 0.005*"baswedan" + 0.005*"dalam"'), (8, '0.021*"anies" + 0.021*"prabowo" + 0.013*"jokowi" + 0.006*"ganjar" + 0.006*"pilpres"'), (9, '0.015*"prabowo" + 0.010*"anies" + 0.010*"indonesia" + 0.008*"anggaran" + 0.005*"rakyat"')]
[[0.         0.         0.         

In [24]:
train = pd.read_excel('../data/cleaned_labeled_dataset_2.xlsx')
train

,text,label,clean_text
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam,kunjungan prabowo meresmikan menyerahkan proye...
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik,anies tepuk tangan meriah rektor mewajibkan ma...
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi,"memang bener memang, pendukung 01 goblok, pend..."
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik,anies bersikap kritis kinerja prabowo dianggap...
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik,anies baswedan harap asn tni polri pegang sump...
...,...,...,...
4995,RT @l1DsGGe2xObT3t72dHwqlT58X7jvEYtEnauZIZSYwS...,Politik,"ngeliat debat kemaren prabowo kicep, kekira ke..."
4996,Masyarakat yakin bahwa Prabowo-Gibran memiliki...,Politik,masyarakat prabowo-gibran memiliki visi sejala...
4997,imo both are irrational but yg satu jauh lebih...,Ekonomi,imo both are irrational but irrational tbh. bu...
4998,@cIIGSdjaPrmAQY1E4gWnLpIZSdyQn8ZMhjJzgOsxfRM= ...,Pertahanan dan Keamanan,"look at that ganjar, berkecimpung lgislatif, e..."


In [31]:
train.dropna(inplace=True)
train.reset_index(drop=True, inplace=True)
train.isnull().sum()

text          0
label         0
clean_text    0
dtype: int64

In [43]:
def get_top_words_train(df, n=3, stop_words=None):
    # Pisahkan data berdasarkan label
    grouped = df.groupby('label')

    words_freq = {}

    for name, group in grouped:
        # Tokenisasi dan hitung frekuensi kata
        vectorizer = CountVectorizer(stop_words=stop_words)
        X = vectorizer.fit_transform(group['clean_text'])
        word_freq = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
        total_freq = word_freq.sum().sort_values(ascending=False)
        
        # Simpan n kata yang paling sering muncul dan frekuensinya
        words_freq[name] = total_freq.nlargest(n).to_dict()

    # buat fitur dari kata-kata yang paling sering muncul
    features = set()
    for label, freq in words_freq.items():
        for word in freq.keys():
            features.add(word)

    features = list(features)

    return words_freq, features

words_freq_kata_train, features_train = get_top_words_train(train, n=100)

# print(words_freq_kata_train)

# simpan jadi dictionary
words_freq_kata_train = dict(words_freq_kata_train)

# cek apakah kata-kata yang paling sering muncul di data latih ada di data uji, lalu simpan jika ada
words_freq_kata_test = {}
for word in features:
    if word in features_train:
        words_freq_kata_test[word] = words_freq_kata[word]

# simpan jadi file

with open('top_words_test_2.json', 'w') as f:
    json.dump(words_freq_kata_test, f)
